In [ ]:
# setup paths
import sys
basedir = '/Users/nickm/thesis/icml2023paper/1d_euler'
readwritedir = '/Users/nickm/thesis/icml2023paper/1d_euler'

sys.path.append('{}/core'.format(basedir))
sys.path.append('{}/simulate'.format(basedir))
sys.path.append('{}/ml'.format(basedir))

In [ ]:
# import external packages
import jax
import jax.numpy as jnp
import numpy as onp
from jax import config, vmap
config.update("jax_enable_x64", True)
import xarray
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from initialconditions import get_a0, f_init_lax_shock_tube, shock_tube_problem_1, shock_tube_problem_2, shock_tube_problem_3, shock_tube_problem_4, shock_tube_problem_5, shock_tube_problem_6, shock_tube_problem_7, shock_tube_problem_8
from simparams import CoreParams, SimulationParams
from simulations import ShockTubeSim
from helper import get_rho, get_u, get_p, get_c
from trajectory import get_trajectory_fn, get_inner_fn

In [ ]:
def get_core_params(Lx = 1.0, gamma = 5/3, bc = 'periodic', fluxstr = 'laxfriedrichs'):
    return CoreParams(Lx, gamma, bc, fluxstr)


def get_sim_params(name = "test", cfl_safety=0.3, rk='ssp_rk3'):
    return SimulationParams(name, basedir, readwritedir, cfl_safety, rk)


def plot_a(a, core_params, mins = [0.0 - 2e-2] * 3, maxs= [1.0 + 5e-2] * 3):
    x = jnp.linspace(0.0, core_params.Lx, a.shape[1])
    
    fig, axs = plt.subplots(1, 3, figsize=(11, 3))
    axs[0].plot(x, get_rho(a, core_params))
    axs[0].set_title(r'$\rho$')
    axs[0].set_ylim([mins[0], maxs[0]])
    
    axs[1].plot(x, get_u(a, core_params))
    axs[1].set_title(r'$u$')
    axs[1].set_ylim([mins[1], maxs[1]])
    
    axs[2].plot(x, get_p(a, core_params))
    axs[2].set_title(r'$p$')
    axs[2].set_ylim([mins[2], maxs[2]])

def plot_ac(a, core_params, mins = [0.0 - 2e-2] * 3, maxs= [1.0 + 5e-2] * 3):
    x = jnp.linspace(0.0, core_params.Lx, a.shape[1])
    
    fig, axs = plt.subplots(1, 4, figsize=(11, 3))
    axs[0].plot(x, get_rho(a, core_params))
    axs[0].set_title(r'$\rho$')
    axs[0].set_ylim([mins[0], maxs[0]])
    
    axs[1].plot(x, get_u(a, core_params))
    axs[1].set_title(r'$u$')
    axs[1].set_ylim([mins[1], maxs[1]])
    
    axs[2].plot(x, get_p(a, core_params))
    axs[2].set_title(r'$p$')
    axs[2].set_ylim([mins[2], maxs[2]])
    
    axs[3].plot(x, get_c(a, core_params))
    axs[3].set_title(r'$c$')
    axs[3].set_ylim([mins[1], maxs[1]])

def plot_trajectory(trajectory, core_params, mins = [0.0 - 2e-2] * 3, maxs= [1.0 + 5e-2] * 3):
    nx = trajectory.shape[2]
    xs = jnp.arange(nx) * core_params.Lx / nx
    xs = xs.T.reshape(-1)
    coords = {
        'x': xs,
        'time': t_inner * jnp.arange(outer_steps)
    }
    rhos = trajectory[:, 0, :]
    xarray.DataArray(rhos, dims=["time", "x"], coords=coords).plot(
        col='time', col_wrap=5)

    us = trajectory[:, 1, :] / trajectory[:, 0, :]
    xarray.DataArray(us, dims=["time", "x"], coords=coords).plot(
        col='time', col_wrap=5)

    ps = (core_params.gamma - 1) * (trajectory[:, 2, :] - 0.5 * trajectory[:, 1, :]**2 / trajectory[:, 0, :])
    xarray.DataArray(ps, dims=["time", "x"], coords=coords).plot(
        col='time', col_wrap=5)

### Shock Tube Problem 1

JE2: http://ammar-hakim.org/sj/je/je2/je2-euler-shock.html

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 1.4, 'bc': 'ghost', 'fluxstr': 'musclcharacteristic'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.2, 'rk' : 'ssp_rk3'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = ShockTubeSim(core_params, sim_params)

nx = 200
f_init = shock_tube_problem_1(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 0.05
outer_steps = 5

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
maxs = [1.1, 1.5, 1.1]
plot_ac(a0, core_params, maxs=maxs)
plot_ac(trajectory[4], core_params, maxs=maxs)

### Problem 2

http://ammar-hakim.org/sj/je/je2/je2-euler-shock.html

MUSCL gives NaNs for this problem.

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 1.4, 'bc': 'ghost', 'fluxstr': 'muscl'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.1, 'rk' : 'ssp_rk3'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = ShockTubeSim(core_params, sim_params)

nx = 100
f_init = shock_tube_problem_2(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 0.0015
outer_steps = 10

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
maxs = [1.1, 2.1, 0.5]
mins = [-0.05, -2.1, -0.05]
plot_a(a0, core_params, maxs=maxs, mins=mins)
plot_a(trajectory[1], core_params, maxs=maxs, mins=mins)

### Problem 3

The 1D Noh problem. MUSCL gives NaNs once again.

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 5/3, 'bc': 'ghost', 'fluxstr': 'muscl'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.2, 'rk' : 'ssp_rk3'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = ShockTubeSim(core_params, sim_params)

nx = 100
f_init = shock_tube_problem_3(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 1.0
outer_steps = 2

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
maxs = [1.1, 1.5, 1.1]
plot_a(a0, core_params, maxs=maxs)
plot_a(trajectory[-1], core_params, maxs=maxs)

### Problem 4

MUSCL fails on this problem.

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 1.4, 'bc': 'ghost', 'fluxstr': 'muscl'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.2, 'rk' : 'ssp_rk3'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = ShockTubeSim(core_params, sim_params)

nx = 100
f_init = shock_tube_problem_4(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 0.012
outer_steps = 2

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
maxs = [1.1, 1.5, 1.1]
plot_a(a0, core_params, maxs=maxs)
plot_a(trajectory[-1], core_params, maxs=maxs)

### Problem 5

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 1.4, 'bc': 'ghost', 'fluxstr': 'muscl'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.2, 'rk' : 'ssp_rk3'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = ShockTubeSim(core_params, sim_params)

nx = 100
f_init = shock_tube_problem_5(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 0.035
outer_steps = 2

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
maxs = [35.0, 20.5, 1800.0]
mins = [-1.0, -10.0, -1.0]
plot_a(a0, core_params, maxs=maxs, mins=mins)
plot_a(trajectory[-1], core_params, maxs=maxs, mins=mins)

### Problem 6

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 1.4, 'bc': 'ghost', 'fluxstr': 'muscl'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.2, 'rk' : 'ssp_rk3'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = ShockTubeSim(core_params, sim_params)

nx = 100
f_init = shock_tube_problem_6(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 1.0
outer_steps = 3

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
maxs = [1.5, 1.5, 1.1]
plot_a(a0, core_params, maxs=maxs)
plot_a(trajectory[-1], core_params, maxs=maxs)

### Problem 7: Density Propagation

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 1.4, 'bc': 'periodic', 'fluxstr': 'muscl'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.2, 'rk' : 'ssp_rk3'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = ShockTubeSim(core_params, sim_params)

nx = 100
f_init = shock_tube_problem_7(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 1.0
outer_steps = 3

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
maxs = [1.5, 1.5, 1.1]
plot_a(a0, core_params, maxs=maxs)
plot_a(trajectory[-1], core_params, maxs=maxs)

### Problem 8

MUSCL fails (NaNs)

In [ ]:
kwargs_core_params = {'Lx': 1.0, 'gamma': 1.4, 'bc': 'ghost', 'fluxstr': 'muscl'}
kwargs_sim = {'name' : "test_euler", 'cfl_safety' : 0.2, 'rk' : 'ssp_rk3'}

core_params = get_core_params(**kwargs_core_params)
sim_params = get_sim_params(**kwargs_sim)
sim = ShockTubeSim(core_params, sim_params)

nx = 800
f_init = shock_tube_problem_8(core_params)
a0 = get_a0(f_init, core_params, nx)

# simulate for some time
t_inner = 0.0039
outer_steps = 2

inner_fn = get_inner_fn(sim.step_fn, sim.dt_fn, t_inner)
trajectory_fn = get_trajectory_fn(inner_fn, outer_steps)
trajectory = trajectory_fn(a0)

In [ ]:
maxs = [40.0, 14.0, 800.0]
plot_a(a0, core_params, maxs=maxs)
plot_a(trajectory[-1], core_params, maxs=maxs)